In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv(r"E:\Data Science by SRK\Machine_learning\Classification\mouse_data\cleaned_dataset_mouse.csv")

In [3]:
df

,Med_1_mL,Med_2_mL,Virus_Present
0,6.508231,8.582531,0
1,4.126116,3.073459,1
2,6.427870,6.369758,0
3,3.672953,4.905215,1
4,1.580321,2.440562,1
...,...,...,...
395,2.884122,3.271748,1
396,7.290855,9.488672,0
397,7.895325,8.272529,0
398,2.690592,2.674979,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Med_1_mL       400 non-null    float64
 1   Med_2_mL       400 non-null    float64
 2   Virus_Present  400 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 9.5 KB


In [5]:
df.isnull().sum()

Med_1_mL         0
Med_2_mL         0
Virus_Present    0
dtype: int64

# X and y

In [6]:
X = df.drop(columns = ['Virus_Present'])
y = df['Virus_Present']

# Train_test_split

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= True)

# ADABOOST
**Applying Hyperparameter Tuning for identifying best parameters for adaboost**

In [8]:
from sklearn.model_selection import GridSearchCV

# modelling

from sklearn.ensemble import AdaBoostClassifier
estimator_ab = AdaBoostClassifier()

# parameters grid
param_grid_ab = {"n_estimators" : list(range(1,100))}

# grid search
grid_ab = GridSearchCV(estimator_ab, param_grid_ab, cv = 5, scoring = 'accuracy')
grid_ab.fit(X_train, y_train)

# best parameters for adaboost model
grid_ab.best_params_

{'n_estimators': 2}

# ADA Boost model important features

In [9]:
feats_ab = pd.DataFrame(data = grid_ab.best_estimator_.feature_importances_,
                       index = X.columns, 
                       columns = ['Importance'])
imp_features_list_ab = feats_ab[feats_ab['Importance']>0].index.to_list()
imp_features_list_ab

['Med_1_mL', 'Med_2_mL']

# ADABoost Classifier

**with imortant features and best hyperparameters**

In [10]:
# important input variables
X_imp = X[imp_features_list_ab]

# train test split
X_train_ab, X_test_ab, y_train_ab, y_test_ab = train_test_split(X_imp, y, test_size = 0.2, random_state = True)

# modelling
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators= 2)
ada.fit(X_train_ab, y_train_ab)

# Evaluation on train data
ypred_train_ab = ada.predict(X_train_ab)

from sklearn.metrics import accuracy_score
print("Train_accuracy : ", accuracy_score(y_train_ab, ypred_train_ab))

# cross validation
from sklearn.model_selection import cross_val_score
print("cross_val_score : ", cross_val_score(ada, X_train_ab, y_train_ab, cv = 5).mean())

# Evaluation
ypred_test_ab = ada.predict(X_test_ab)
print("Test_accuracy : ", accuracy_score(y_test_ab, ypred_test_ab))


Train_accuracy :  1.0
cross_val_score :  0.996875
Test_accuracy :  1.0


In [11]:
%%timeit
ada.fit(X_train_ab, y_train_ab)


10.3 ms ± 336 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
